In [89]:
import pandas as pd
import numpy as np
import scipy.stats as sc


In [104]:
# get data from https://www.football-data.co.uk/
data = pd.read_csv(f'https://www.football-data.co.uk/mmz4281/1516/E0.csv')

#selecting columns from dataframe to work with
data = data[['Date','HomeTeam','AwayTeam','FTHG','FTAG']]


In [105]:
# number of league games for a full season and print of all teams in league to copy and paste 
lg_games = len(data['HomeTeam'].unique()) - 1
data['HomeTeam'].unique()


array(['Bournemouth', 'Chelsea', 'Everton', 'Leicester', 'Man United',
       'Norwich', 'Arsenal', 'Newcastle', 'Stoke', 'West Brom',
       'Aston Villa', 'Southampton', 'Sunderland', 'Swansea', 'Tottenham',
       'Watford', 'West Ham', 'Crystal Palace', 'Man City', 'Liverpool'],
      dtype=object)

In [106]:

class Goal_Exp:
    
    def __init__(self, home, away, num_games, df):
        self.home = home
        self.away = away
        self.num_games = num_games
        self.df =  df
        
    # calculate average goals of all league two arguments of a data frame and num of games returns a lists
    def calc_lge_avg(self):
        team_count = len(self.df['HomeTeam'].unique())
        games = self.df.tail(self.num_games * team_count)
        homeMean = games['FTHG'].mean()
        awayMean = games['FTAG'].mean()
        return [homeMean, awayMean]
    
    # returns both teams last number of games in a list of dataframes
    def team_last_games(self):
        games = []
        games.append(self.df[(self.df['HomeTeam']==self.home)].tail(self.num_games))
        games.append(self.df[(self.df['AwayTeam']==self.away)].tail(self.num_games))
        return games
    
    # get teams attacking and defensive strengths as a list value at index 1 is attacking strengths
    # and defensive strength at index 2
    def team_strengths(self):
        df2 = self.team_last_games()
        lg_avg = self.calc_lge_avg()
        tm_avg = 0
        stList = []
        stList.append((df2[0]['FTHG'].mean()) / lg_avg[0])
        stList.append((df2[0]['FTAG'].mean()) / lg_avg[1])
        stList.append((df2[1]['FTAG'].mean()) / lg_avg[1])
        stList.append((df2[1]['FTHG'].mean()) / lg_avg[0])
    
        return stList
    
    # calculate each teams expected goals
    def exp_goals(self):
        strengths = self.team_strengths()
        lg_avg = self.calc_lge_avg()
    
        home_exp = round(strengths[0] * strengths[3] * lg_avg[0], 3)
        away_exp = round(strengths[2] * strengths[1] * lg_avg[1], 3)
    
        return [home_exp, away_exp]
    
    # calculates implied probability scoring an certain amount of goals using possion Probability mass function
    def goals_prob(self):
        poss = []
        goals = range(7)
        expected = self.exp_goals()
        for i in expected:
            if expected.index(i) == 0:
                aList = []
                for g in goals:
                    aList.append(sc.poisson.pmf(g, i))
            elif expected.index(i) == 1:
                aList = []
                for g in goals:
                    aList.append(sc.poisson.pmf(g, i))
            poss.append(aList)
        return poss
    # creates a dataframe with probabilties of correct score calulated by multiply
    # the Poisson probability for each specific team to score a number of goals
    def scores_table(self):
        gls = self.goals_prob()
        col= [['Possion num of gls'],[],[],[],[],[],[],[]]
        index = 0
        for lst in gls:
            if gls.index(lst) == 0:
                for e in lst:
                    col[index + 1].append(round((e *100), 2))
                    index +=1
            elif gls.index(lst) == 1:
                index = 0
                for e in lst:
                    col[0].append(round((e *100), 2))
                    index +=1
        index = 1            
        for i in gls[0]:
            for j in range(len(gls[1])):
                col[index].append(round((i * gls[1][j])* 100, 2))
            index += 1
        df = pd.DataFrame(col,
                         columns = [f'{self.away} goals', '0', '1', '2', '3', '4', '5', '6' ],
                         index = [f'{self.home} goals', '0', '1', '2', '3', '4', '5', '6' ])
        return df
        
    
    # calaculate implied probability of each score by multiplying each teams goals probabilties
    def calc_prob(self):
        new_list = [[],[],[],[],[],[]]
        home_list = self.goals_prob()[0]
        away_list = self.goals_prob()[1]
        index = 0
        for i in home_list:
            for j in range(len(away_list)):
                home_ind = home_list.index(home_list[index])
                away_ind = away_list.index(away_list[j])
            
                if home_ind > away_ind:
                    new_list[0].append((i * away_list[j]) * 100)
                    new_list[3].append(str(home_ind) + "-" + str(away_ind))
                elif home_ind < away_ind:
                    new_list[1].append((i * away_list[j]) * 100)
                    new_list[4].append(str(home_ind) + "-" + str(away_ind))
                else:
                    new_list[2].append((i * away_list[j]) * 100)
                    new_list[5].append(str(home_ind) + "-" + str(away_ind))
                
            index += 1
            
        return new_list
    
    # calculate Implied probability of match result odds
    def match_imp_odds(self):
        odds = self.calc_prob()
        home_odd  = 1/((sum(odds[0])/100))
        away_odd = 1/((sum(odds[1])/100))
        draw_odd = 1/((sum(odds[2])/100))
        return [round(home_odd, 3), round(draw_odd, 3), round(away_odd, 3)]
    
    # returns list of all data ready to append to a dataframe
    def data_row(self):
        lg_avg = self.calc_lge_avg()
        last = self.team_last_games()
        strg = self.team_strengths()
        exp = self.exp_goals()
        odds = self.match_imp_odds()
        return [
            self.num_games,
            self.home,
            last[0]['FTHG'].sum(),
            last[0]['FTAG'].sum(),
            round(strg[0], 3),
            round(strg[1], 3),
            exp[0],
            self.away,
            last[1]['FTHG'].sum(),
            last[1]['FTAG'].sum(),
            round(strg[2], 3),
            round(strg[3], 3),
            exp[1],
            odds[0],
            odds[1],
            odds[2]
            
        ]
    
    
    
   

In [107]:
g1 = Goal_Exp('Tottenham', 'Stoke', 19, data)

In [114]:
g1.scores_table()

,Stoke goals,0,1,2,3,4,5,6
Tottenham goals,Possion num of gls,52.00,34.01,11.12,2.42,0.40,0.05,0.01
0,13.35,6.94,4.54,1.48,0.32,0.05,0.01,0.00
1,26.88,13.98,9.14,2.99,0.65,0.11,0.01,0.00
2,27.07,14.07,9.20,3.01,0.66,0.11,0.01,0.00
3,18.17,9.45,6.18,2.02,0.44,0.07,0.01,0.00
4,9.15,4.76,3.11,1.02,0.22,0.04,0.00,0.00
5,3.69,1.92,1.25,0.41,0.09,0.01,0.00,0.00
6,1.24,0.64,0.42,0.14,0.03,0.00,0.00,0.00


In [115]:
g1.calc_lge_avg()

[1.4921052631578948, 1.2078947368421054]

In [116]:
g1.team_strengths()

[1.2345679012345678, 0.6535947712418301, 0.8278867102396513, 1.09347442680776]

In [117]:
g1.exp_goals()

[2.014, 0.654]

In [118]:
 print("match odds >>> home : " + str(g1.match_imp_odds()[0]) + ", draw : " + str(g1.match_imp_odds()[1]) + ", away : " + str(g1.match_imp_odds()[2]))  
    

match odds >>> home : 1.451, draw : 5.111, away : 9.062


In [119]:
g1.data_row()

[19,
 'Tottenham',
 35,
 15,
 1.235,
 0.654,
 2.014,
 'Stoke',
 31,
 19,
 0.828,
 1.093,
 0.654,
 1.451,
 5.111,
 9.062]